In [96]:
import pandas as pd
import keras
from keras.engine import Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
import numpy as np
from keras.preprocessing import image
from keras_vggface import utils
import glob, re

In [97]:
model = VGGFace(model = "senet50", include_top=False, input_shape=(300, 300, 3), pooling='avg') # pooling: None, avg or max

In [106]:
train_label_EI = pd.read_csv("train_E_I.csv", usecols=[1], delimiter=',').to_numpy().flatten()
test_label_EI = pd.read_csv("test_E_I.csv", usecols=[1], delimiter=',').to_numpy().flatten()

train_label_MBTI = pd.read_csv("train_MBTI.csv", usecols=[1], delimiter=',').to_numpy().flatten()
test_label_MBTI = pd.read_csv("test_MBTI.csv", usecols=[1], delimiter=',').to_numpy().flatten()

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_label_MBTI = le.fit_transform(train_label_MBTI)
test_label_MBTI = le.transform(test_label_MBTI)

[14 14 14 ...  2  2  2] [14 14 14 14 14 14 14 14 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8 10 10 10 15 15 15 15 15 15 15 15 15 15 15 15 15 13 13 13 13 13 13
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9 11 11 11
 11 11 11 11 11 11 11 11 11 11  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  3  3  3  3  3

In [107]:
def preprocess_image(path):
    img = image.load_img(path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    return x

In [108]:
df_train_EI = pd.read_csv("train_E_I.csv", usecols=[0], delimiter=',').values.flatten()
df_test_EI = pd.read_csv("test_E_I.csv", usecols=[0], delimiter=',').values.flatten()
preprocessed_train_images_EI = np.vstack([preprocess_image(df_train_EI[i]) for i in range(df_train_EI.shape[0])])
preprocessed_test_images_EI = np.vstack([preprocess_image(df_test_EI[i]) for i in range(df_test_EI.shape[0])])

df_train_MBTI = pd.read_csv("train_MBTI.csv", usecols=[0], delimiter=',').values.flatten()
df_test_MBTI = pd.read_csv("test_MBTI.csv", usecols=[0], delimiter=',').values.flatten()
preprocessed_train_images_MBTI = np.vstack([preprocess_image(df_train_MBTI[i]) for i in range(df_train_MBTI.shape[0])])
preprocessed_test_images_MBTI = np.vstack([preprocess_image(df_test_MBTI[i]) for i in range(df_test_MBTI.shape[0])])

In [109]:
vgg_train_features_EI = model.predict(preprocessed_train_images_EI)
vgg_train_features_EI = vgg_train_features_EI.reshape(train_label_EI.shape[0], -1)

vgg_test_features_EI = model.predict(preprocessed_test_images_EI)
vgg_test_features_EI = vgg_test_features_EI.reshape(test_label_EI.shape[0], -1)

vgg_train_features_MBTI = model.predict(preprocessed_train_images_MBTI)
vgg_train_features_MBTI = vgg_train_features_MBTI.reshape(train_label_MBTI.shape[0], -1)

vgg_test_features_MBTI = model.predict(preprocessed_test_images_MBTI)
vgg_test_features_MBTI = vgg_test_features_MBTI.reshape(test_label_MBTI.shape[0], -1)

In [110]:
np.save("vgg_feature_train_EI.npy", vgg_train_features_EI)
np.save("vgg_feature_test_EI.npy", vgg_test_features_EI)
np.save("label_train_EI.npy", train_label_EI)
np.save("label_test_EI.npy", test_label_EI)

np.save("vgg_feature_train_MBTI.npy", vgg_train_features_MBTI)
np.save("vgg_feature_test_MBTI.npy", vgg_test_features_MBTI)
np.save("label_train_MBTI.npy", train_label_MBTI)
np.save("label_test_MBTI.npy", test_label_MBTI)